## 테마주 수집하기

https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=227

In [ ]:
# 라이브러리 로드
import pandas as pd

In [ ]:
# 시각화를 위한 폰트설정
import platform
import matplotlib.pyplot as plt

system_name = platform.system()

if system_name == "Darwin" :
    font_family = "AppleGothic"
elif system_name == "Windows":
    font_family = "Malgun Gothic"
else:
    # Linux
    !apt -qq -y install fonts-nanum > /dev/null
    import matplotlib.font_manager as fm

    fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    font = fm.FontProperties(fname=fontpath, size=9)
    fm._rebuild()
    font_family = "NanumBarunGothic"
    
# 폰트설정
plt.rc("font", family=font_family)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# 그래프 스타일 설정
plt.style.use("ggplot")


# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
pd.Series([1, 3, 5, -7, 9]).plot(title="한글폰트 확인")

In [ ]:
# 수집할 테마 주소
url = "https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=227"

In [ ]:
# 해당 테마 읽어오기
table = pd.read_html(url, encoding="cp949")
len(table)

In [ ]:
df = table[2].dropna(how="all")
df = df.dropna(axis=1)
df.shape

In [ ]:
import datetime

today = datetime.datetime.today()
today = today.strftime("%Y-%m-%d")
today

In [ ]:
df["조회일자"] = today

In [ ]:
df.head()

In [ ]:
vol = df[["종목명", "거래량"]]
vol = vol.set_index("종목명")
vol.sort_values(by=["거래량"]).tail(30).plot.barh(figsize=(10, 15))

In [ ]:
price = df[["종목명", "거래대금"]]
price = price.set_index("종목명")
price.sort_values(by=["거래대금"]).tail(30).plot.barh(figsize=(10, 15))

## 종목코드 수집
* 종목코드는 read_html 로 수집이 되지 않습니다.
* 그래서 직접 수집을 합니다.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
response = requests.get(url)

In [ ]:
html = bs(response.text, "lxml")

In [ ]:
table_2 = html.select("table")[2]

In [ ]:
a_tags = table_2.select("a")

In [ ]:
a_tags[0]["href"].split("=")[-1]

In [ ]:
item_list = []
for a_tag in a_tags:
    item_code = a_tag["href"].split("=")[-1]
    item_list.append(item_code)

In [ ]:
df.shape

In [ ]:
item_code_list = pd.Series(item_list).drop_duplicates()
item_code_list = item_code_list.tolist()

In [ ]:
len(item_code_list)

In [ ]:
item_code_list.remove('javascript:;')
item_code_list

In [ ]:
df["종목코드"] = item_code_list

In [ ]:
df

## 종목상세정보

'종목코드', '시가총액', '시가총액순위', '상장주식수', '액면가l매매단위', '외국인한도주식수(A)',
'외국인보유주식수(B)', '외국인소진율(B/A)', '투자의견l목표주가', '52주최고l최저',
'추정PERlEPS', '동일업종 PER', '동일업종 등락률', '배당수익률'

In [ ]:
item_url = f"https://finance.naver.com/item/main.nhn?code={item_code}"
item_url

In [ ]:
tables = pd.read_html(item_url, encoding="cp949")
len(tables)

In [ ]:
tables

In [ ]:
tables[5]

### 컬럼명과 값 형태로 되어 있는 테이블만 수집

In [ ]:
# 컬럼의 수가 2인 테이블만 따로 list로 생성
item_info = []
for table in tables:
    if table.shape[1] == 2:
        item_info.append(table)

In [ ]:
# 리스트의 값 확인
item_info

### concat을 통해 데이터를 병합

<img src="https://pandas.pydata.org/pandas-docs/stable/_images/merging_concat_keys.png">

In [ ]:
df_item = pd.concat(item_info)
df_item

### transpose를 통해 행과 열의 위치를 변경

In [ ]:
# transpose는 .transpose() 혹은  .T 로 사용해 보실 수 있습니다.
df_item.set_index(0).T

### 개별 주가 정보를 수집하는 함수를 생성

In [ ]:
import time

def get_item_info(item_code):
    item_url = f"https://finance.naver.com/item/main.nhn?code={item_code}"
    tables = pd.read_html(item_url, encoding="cp949")
    # 컬럼의 수가 2인 테이블만 따로 list로 생성
    item_info = []
    for table in tables:
        if table.shape[1] == 2:
            item_info.append(table)
    df_item = pd.concat(item_info)
    df_item = df_item.set_index(0).T
    df_item["종목코드"] = item_code
    time.sleep(0.5)
    return df_item

In [ ]:
get_item_info(item_code)

## tqdm은?
* 오래 걸리는 작업의 진행 게이지를 표시
* https://tqdm.github.io/

In [ ]:
from tqdm import tqdm, tqdm_pandas

tqdm_pandas(tqdm)

In [ ]:
result = df["종목코드"].progress_apply(get_item_info)

In [ ]:
df_item_info = pd.concat(result.tolist())

### merge 를 통한 데이터프레임 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_axis1_join_axes.png">

In [ ]:
df_info = df.merge(df_item_info)

In [ ]:
df_info.shape

In [ ]:
df_info.head()

### 일부 컬럼만 가져옴

In [ ]:
df_info.columns

In [ ]:
info_cols = ['종목명', '시가총액', '시가총액순위', '현재가', '전일비', '등락률', '매수호가', '매도호가', '거래량', '거래대금', '전일거래량',
       '조회일자', '종목코드', '상장주식수', '액면가l매매단위', '외국인한도주식수(A)',
       '외국인보유주식수(B)', '외국인소진율(B/A)', '투자의견l목표주가', '52주최고l최저',
       '동일업종 PER', '동일업종 등락률', '배당수익률']

In [ ]:
df_info[info_cols].sort_values("시가총액순위")

### 파일명 만들기

In [ ]:
file_name = url.split("type=")[-1].replace("&", "_").replace("=", "_")
file_name = f"{file_name}.csv"

In [ ]:
df_info.to_csv(file_name, index=False)

In [ ]:
pd.read_csv(file_name)

* 참고 : 구글드라이브 마운트를 통해 내 구글 드라이브에 파일을 저장하고 불러오기
    * [외부 데이터: 로컬 파일, 드라이브, 스프레드시트, Cloud Storage - Colaboratory](https://colab.research.google.com/notebooks/io.ipynb)